In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

In [2]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.join_data_preparation import JoinDataPreparator
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')
from DataPrepare.join_data_preparation import JoinDataPreparator
hdf_path = "/home/ziniu.wzn/imdb-benchmark/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [3]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
def train_all(algo="chow-liu", max_parents=1):
    for i, relationship_obj in enumerate(schema.relationships):
        sample_size = 10000000
        relation = relationship_obj.identifier
        df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                        sample_size, relationship_list=[relation], post_sampling_factor=10)
        print(full_join_est)
        print(len(df), len(df.columns))
        print(meta_types)
        meta_info = build_meta_info(df.columns, null_values)
        bn = Pgmpy_BN(relation, meta_info, full_join_est)
        model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl"
        print(model_path)
        bn.build_from_data(df, algorithm=algo, max_parents=max_parents, ignore_cols=['id'], sample_size=200000)
        pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        """
        try:
            BN.model = Bn.model.to_junction_tree()
            model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_junction.pkl"
            pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        except:
            print("This BN is not able to transform into junction tree, probably because it's not connected, just use BN")
        """

In [4]:
#train_all("chow-liu", 3)

In [5]:
from Evaluation.parse_query_imdb import prepare_join_queries
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
ensemble_location = "../Benchmark/IMDB/ensemble_loader.pkl"
#query_filename = "/home/ziniu.wzn/imdb-benchmark/cardinality/job_all.sql"
query_filename = "../Benchmark/IMDB/job-light.sql"
pairwise_rdc_path = "../Benchmark/IMDB/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=False, true_card_exist=True)

In [6]:
from Models.BN_ensemble_model import BN_ensemble
bn_ensemble = BN_ensemble(schema)
algo = "chow-liu"
distinct_mapping = dict()
distinct_mapping['movie_keyword.keyword_id']={117: 8, 8200: 10, 398: 5, 7084: 20}
distinct_mapping['movie_companies.company_id']={22956: 62}
for i in range(5):
    with open(f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl", "rb") as f:
        bn = pickle.load(f)
        bn.infer_algo = "exact-jit"
        bn.init_inference_method()
        bn.n_distinct_mapping = distinct_mapping
    bn_ensemble.bns[i] = bn

In [7]:
queries = bn_ensemble.parse_query_all(parsed_queries)

In [8]:
import time
tic = time.time()
a = bn_ensemble.cardinality(queries[15])
print(a)
print(time.time() - tic)

11113.345332324363
0.004244565963745117


In [9]:
with open("../Benchmark/IMDB/job-light.sql", "rb") as f:
    real_query = f.readlines()

In [10]:
import time
latency = []
q_error = []
for i, q in enumerate(queries):
    tic = time.time()
    pred = bn_ensemble.cardinality(q, sample_size=10000, hard_sample=False)
    #try:
     #   pred = bn_ensemble.cardinality(q, sample_size=10000, hard_sample=False)
    #except:
     #   print("invalid")
        #this query itself is invalid or it is not recognizable by the learnt BN
      #  continue
    print(i, real_query[i])
    latency.append(time.time()-tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred/true[i], true[i]/pred)
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_error.append(error)

0 b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2||715\n'
true cardinality 715, predicted 3536.0838012929034 with q-error 4.9455717500600045
1 b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010||9\n'
true cardinality 9, predicted 1.9135314657190496 with q-error 4.703345704648792
2 b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2 AND t.production_year>2010||47\n'
true cardinality 47, predicted 13.748553492837658 with q-error 3.418541450523123
3 b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_

46 b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100||16461908\n'
true cardinality 16461908, predicted [35721697.4243473] with q-error [2.16996094]
47 b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci WHERE t.id=mi.movie_id AND t.id=mc.movie_id AND t.id=ci.movie_id AND ci.role_id=2 AND mi.info_type_id=16 AND t.production_year>2005 AND t.production_year<2009||24675801\n'
true cardinality 24675801, predicted 19048362.51158478 with q-error 1.2954289894993725
48 b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci WHERE t.id=mi.movie_id AND t.id=mc.movie_id AND t.id=ci.movie_id AND ci.role_id=2 AND mi.info_type_id=16||132317651\n'
true cardinality 132317651, predicted [1.56389873e+08] with q-error [1.18192752]
49 b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci WHER

In [11]:
for i in [50, 90, 95, 100]:
    print(np.percentile(q_error, i))
print(np.mean(latency))

1.287723277784334
3.5334621924040626
4.8365700296249585
19.1399934612504
0.005419196401323591


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
for i in [50, 90, 95, 100]:
    print(np.percentile(q_error, i))
print(np.mean(latency))

In [ ]:
np.asarray(np.asarray(1.0))

In [ ]:
np.ones(3)[[1]]